# Основы разработки под sklearn
---
С.Ю. Папулин (papulin.study@yandex.ru)

### Содержание

- [Общие сведения](#Общие-сведения)
- [Реализация модели предсказания](#Реализация-модели-предсказания)
- [Реализация транформации](#Реализация-транформации)
- [Применение `Pipeline`](#Применение-Pipeline)
- [Источники](#Источники)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Общие сведения

Объекты `sklearn` и их методы:
- **Estimator**: `fit` и `partial_fit` (дообучение)
- **Predictor**: `predict` + для классификации дополнительно `decision_function` и/или `predict_proba`
- **Transformer**: `transform` и `fit_transform`
- **Model**: `score`

Аргументы методов:
- `fit(X, y, **kwargs) -> self`
- `partial_fit(X, y, **kwargs) -> self`
- `set_params(*args, **kwargs)` и `get_params(deep=True) -> dict`
- `score(X, y, **kwargs) -> float`
- `transform(X, **kwargs) -> X_t`
- `fit_transform(X, y, **kwargs) -> X_t`

`X` - массив размера (n_samples, n_features), `y` - массив размера (n_samples,)

Результат оценки (обучения):
- `coef_`, `idf_` и пр.

Перезаписываются каждый раз после вызова `fit`

## Реализация модели предсказания

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

In [ ]:
help(BaseEstimator)

In [ ]:
help(RegressorMixin)

In [ ]:
class CustomLinearRegression(BaseEstimator, RegressorMixin):
    
    def __init__(self, method='ols'):
        self.method = method
    
    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.n_features_in_ = X.shape[1]
        X_ = np.c_[np.ones(X.shape[0]), X]
        # Вариант 1. не пройдет тесты check_estimator
        self.coef_ = np.linalg.inv(X_.T @ X_) @ X_.T @ y
        # Вариант 2. пройдет тесты check_estimator
        # self.coef_ = np.linalg.pinv(X_) @ y
        return self
    
    def predict(self, X):
        check_is_fitted(self, 'coef_')
        X = check_array(X)
        X_ = np.c_[np.ones(X.shape[0]), X]
        return X_ @ self.coef_

Доступ к параметрам

In [ ]:
model = CustomLinearRegression()
# model.get_params(deep=True)

In [ ]:
# model.set_params(method='gd')

Проверка на совместимость с `sklearn`

In [ ]:
from sklearn.utils.estimator_checks import check_estimator
from sklearn.base import is_regressor 

In [ ]:
# Если есть проблемы с нижележащей командой, обновите threadpoolctl
# %pip install threadpoolctl==3.1.0

In [ ]:
try:
    check_estimator(estimator=CustomLinearRegression())
except Exception as e:
    print(e)

In [ ]:
# Note: Based on _estimator_type
is_regressor(CustomLinearRegression())

Совместимые с `sklearn` объекты можно использовать в `GridSearchCV` для выбора моделей и в `Pipeline` для организации последовательности обработки данных.

### Пример

In [ ]:
def generate_data(n=100, start_x=4, length_x=8, mu=0, sigma=0.5):
    """Генерация данных."""
    from scipy import stats
    f = lambda x: 2 + 0.3*x
    x = stats.uniform.rvs(size=n, loc=start_x, scale=length_x, random_state=1)
    e = stats.norm.rvs(size=n, loc=mu, scale=sigma, random_state=1)
    return x.reshape(-1,1), f(x) + e

In [ ]:
X, y = generate_data()

In [ ]:
# Отображение наблюдений
plt.figure(1, figsize=[4, 4])

plt.subplot(1,1,1)
plt.scatter(X[:,0], y, color="green", label="Sample", zorder=2)
plt.legend()
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
plt.grid(True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Разбиение данных на обучающие и тестовые
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
X_train[:5], y_train[:5]

In [ ]:
model = CustomLinearRegression()

In [ ]:
# Обучение
model.fit(X_train, y_train)
model.coef_

In [ ]:
# Отображение наблюдений и линии регрессии
plt.figure(2, figsize=[4, 4])

xx = np.linspace(X[:,0].min(),X[:,0].max(), 2).reshape(-1,1)

plt.subplot(1,1,1)
plt.scatter(X[:,0], y, color="green", label="Sample", zorder=2)
plt.plot(xx, model.predict(xx), "-", color="grey", label="Regression")
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
plt.legend()
plt.grid(True)

In [ ]:
model.score(X_test, y_test)

Сравнение с реализацией в `sklearn`

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
buildin_model = LinearRegression().fit(X_train, y_train)
buildin_model.intercept_, buildin_model.coef_

In [ ]:
buildin_model.score(X_test, y_test)

## Реализация транформации

In [ ]:
help(TransformerMixin)

In [ ]:
class CustomStandardTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        # TODO(X, y)
        self.params_ = ...
        return self
    
    def transform(self, X):
        # TODO(X, params_)
        X_ = ...
        return X_

## Применение Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# TODO: AddOneTransformer
# TODO: CustomLinearRegression

In [ ]:
pipeline = Pipeline([
    ("addone", AddOneTransformer()),
    ("regressor", CustomLinearRegression())
])

In [ ]:
# Обучение
pipeline.fit(X_train, y_train)

# Параметры модели
print(f'w = {pipeline.named_steps["regressor"].coef_}')

# Качество модели
print(f'R^2 = {pipeline.score(X_test, y_test)}')

# Предсказание
y_test__pred = pipeline.predict(X_test)
y_test__pred[:5]

In [ ]:
# Отображение наблюдений и линии регрессии
plt.figure(2, figsize=[4, 4])

xx = np.linspace(X[:,0].min(),X[:,0].max(), 2).reshape(-1,1)

plt.subplot(1,1,1)
plt.scatter(X[:,0], y, color="green", label="Sample", zorder=2)
plt.plot(xx, pipeline.predict(xx), "-", color="grey", label="Regression")
plt.xlabel("$x$")
plt.ylabel("$f(x)$")
plt.legend()
plt.grid(True)

## Сериализация модели

`joblib`

In [ ]:
from joblib import dump, load

In [ ]:
# Обученная модель
linear_model = LinearRegression().fit(X_train, y_train)
linear_model.intercept_, buildin_model.coef_

In [ ]:
FILE_NAME = 'linear_model.joblib'

In [ ]:
# Сохранение модели (сериализация модели)
dump(linear_model, FILE_NAME) 

In [ ]:
# Там где загружается модель, должен быть
# доступен класс модели
from sklearn.linear_model import LinearRegression

# Загрузка модели (десериализация модели)
linear_model = load(FILE_NAME)

# Проверка
linear_model.intercept_, buildin_model.coef_

## Источники

- [Developing scikit-learn estimators](https://scikit-learn.org/stable/developers/develop.html)
- [Utilities for Developers](https://scikit-learn.org/stable/developers/utilities.html#developers-utils)
- [Glossary of Common Terms and API Elements](https://scikit-learn.org/stable/glossary.html#glossary)
- [A template for scikit-learn contributions](https://github.com/scikit-learn-contrib/project-template)